In [134]:
# Import libraries
import os
import glob
import pandas as pd
import plotly.express as px
import streamlit as st

In [135]:
# Locate the files in the appropriate directory
path = './Adrienne_Data/'
files = glob.glob(path+'/*.csv')

# Load the data and join them
data = []
for file in files:
    df = pd.read_csv(file,index_col=None,header=0)
    data.append(df)
data = pd.concat(data,axis=0,ignore_index=True)

# Clean the data
data.drop(['Unnamed: 0'],axis=1,inplace=True) # Redundant column

In [136]:
# Rename years
# Create a dictionary to map years to their corresponding values
year_mapping = {
    2035: 5, 2040: 10, 2045: 15, 2050: 20,
    2055: 25, 2060: 30, 2065: 35, 2070: 40,
    2075: 45, 2080: 50, 2085: 55, 2090: 60,
    2095: 65, 2100: 70, 2105: 75, 2109: 80
}

# Apply the year mapping using the replace method
data["Year"] = data["Year"].replace(year_mapping)

# Remove the numbers from the metrics column
data['Metric'] = data['Metric'].str.slice(start=2)

# Rename and select metrics
data['Metric'] = data['Metric'].replace({'Employment': 'Jobs',
                                        'LaborIncome':'Labor Income',
                                        'Output':'Total Production Value'})
data = data[data["Metric"].isin(["Jobs", "Labor Income", "Total Production Value"])]

# Rename industries
data['Industry'] = data['Industry'].replace({'TIPU (Transportation, Information, Power and Utilities)':'Transportation and Utilities'})

In [137]:
# data['Metric'] = data['Metric'].replace({'EmployeeCompensation': 'Employee Compensation', 'PropietorsIncome': "Proprietor's Income",'LaborIncome':'Labor Income','OtherPropertyTypeIncome':'Other Property Type Income','IndirectBusinessTaxes':'Indirect Business Taxes','TotalValueAdded':'Total Value Added'})

In [138]:
# Update industry values to group them
industry_mapping = {
    'Service': 'Service and Trade',
    'Trade': 'Service and Trade',
    'Mining': 'Mining, Manufacturing, and Agriculture',
    'Manufacturing': 'Mining, Manufacturing, and Agriculture',
    'Agriculture': 'Mining, Manufacturing, and Agriculture'
}

# Apply the industry mapping using the replace method
data['Industry'] = data['Industry'].replace(industry_mapping)

# Group by 'Industry', 'Year', 'Metric', 'Attribute', and 'Scenario', and then sum the 'Value' column
data = data.groupby(['Industry', 'Year', 'Metric', 'Attribute', 'Scenario'])['Value'].sum().reset_index()

In [139]:
# Group the data by the "Scenario" column
grouped_data = data.groupby('Scenario')

# Create a dictionary to store the separate DataFrames
dataframes_by_scenario = {}

# Iterate through the groups and create separate DataFrames
for scenario, group in grouped_data:
    dataframes_by_scenario[scenario] = group.copy()

# Access the separate DataFrames using the keys "Base Case" and "Higher Receipt"
base_case = dataframes_by_scenario['Base Case']
higher_receipt = dataframes_by_scenario['Higher Receipt']

In [2]:
# Imports
import plotly.graph_objs as go
import pandas as pd
import numpy as np

# data
d = {'x': ['a','b','c','a','b','c','a','b','c'], 'y': [1,2,3,10,20,30,100,200,300], 'group': [1,1,1,2,2,2,3,3,3]}
df = pd.DataFrame(data=d)

# split df by groups and organize them in a dict
groups = df['group'].unique().tolist()
dfs={}
for g in groups:
    dfs[str(g)]=df[df['group']==g]

# get column names from first dataframe in the dict
#colNames = list(dfs[list(dfs.keys())[0]].columns)
#colNames=colNames[:2]


# one trace for each column per dataframe
fig=go.Figure()

# set up the first trace
fig.add_trace(go.Bar(x=dfs['1']['x'],
                             y=dfs['1']['y'],
                             visible=True)
             )
# set up the second trace
fig.add_trace(go.Bar(x=dfs['1']['x'],
                             y=dfs['1']['y'],)
             )

#f=fig.to_dict()

# plotly start
# buttons for menu 1, names
updatemenu=[]
buttons=[]

# button with one option for each dataframe
for df in dfs.keys():
    #print(b, df)
    buttons.append(dict(method='restyle',
                        label=df,
                        visible=True,
                        args=[{'y':[dfs[str(df)]['y'].values],
                               'type':'bar'}, [0]],
                        )
                  )

# another button with one option for each dataframe
buttons2=[]
for df in dfs.keys():
    buttons2.append(dict(method='restyle',
                        label=df,
                        visible=True,
                        args=[{'y':[dfs[str(df)]['y'].values],
                               'type':'bar'}, [1]],
                        )
                  )

# some adjustments to the updatemenus
updatemenu=[]
your_menu=dict()
updatemenu.append(your_menu)
your_menu2=dict()
updatemenu.append(your_menu2)
#updatemenu[1]
updatemenu[0]['buttons']=buttons
updatemenu[0]['direction']='down'
updatemenu[0]['showactive']=True
updatemenu[1]['buttons']=buttons2
updatemenu[1]['y']=0.5

# add dropdown menus to the figure
fig.update_layout(showlegend=False, updatemenus=updatemenu)

# add notations to the dropdown menus
fig.update_layout(
    annotations=[
        go.layout.Annotation(text="<b>group/<br>trace:</b>",
                             x=-0.15, xref="paper",
                             y=1.15, yref="paper",
                             align="left", showarrow=False),
        go.layout.Annotation(text="<b>group/<br>trace:</b>",
                             x=-0.15, xref="paper", y=0.6,
                             yref="paper", showarrow=False),
                  ]
)

fig.show()